# Demo: Few-Shot vid2vid (Talking Head/Face)

給予一張人臉 A 的參考照片，Few-shot vid2vid 能夠將一段影片中的人臉 B 替換成參考照片指定的人臉 A。詳細請看 [README.md](https://github.com/achen353/imaginaire-fsvid2vid/blob/master/README.md)。

<img alt='teaser' src='https://nvlabs.github.io/few-shot-vid2vid/web_gifs/illustration.gif' width='600'/>

## 1. 測試模型運行

成功 build 好 Docker image 並執行後，先測試模型是否能夠正常運行：

### 1.1 測試模型能夠正常執行 inference

In [ ]:
!bash ../scripts/test_inference.sh

### 1.2 測試模型能夠正常執行 training

In [ ]:
!bash ../scripts/test_training.sh

## 2. 模型預測

使用模型進行預測（人臉轉換）時，須先將欲使用的影片（driving video）進行處理，將其轉換成影格照片和相對應的 dlib-68 人臉標記，dlib-68 會在臉部輪廓標記 68 個標記點供模型定位。另外還要準備一張欲使用的人臉照片（reference image）。

### 2.1 準備預測資料

以下整理預測時需要的資料：
1. 影片轉換成的照片 * N （driving video)
2. dlib-68 人臉邊框標記 * N （driving video）
3. 人臉照片 * 1 (reference image）

#### 2.1.1 將欲使用的影片放入 `/src/projects/fs_vid2vid/data/driving/videos`

```
data
└───images（2.1.2 寫入的資料夾）
└───landmarks-dlib68（2.1.3 寫入的資料夾）
└───videos
    └───00000.mp4
    └───00001.mp4
    └───00002.mp4
```

#### 2.1.2 將影片轉換成影格（driving video to images）

In [ ]:
python video_to_frames.py -i projects/fs_vid2vid/data/driving/videos/00000.mp4 -o projects/fs_vid2vid/data/driving/images/00000

#### 2.1.3 準備臉部輪廓標記（dlib-68 landmarks）

In [ ]:
python facial_landmarks.py -i projects/fs_vid2vid/data/driving/images/00000 -o projects/fs_vid2vid/data/driving/landmarks-dlib68/00000

#### 2.1.4 將上面寫入 `/images` 和 `/landmarks-dlib68` 的測試資料放入 `/src/projects/fs_vid2vid/test_data/faceForensics/driving`。

```
faceForensics
└───reference
    ...
└───driving
    └───images
        └───00000
            └───00001.jpg
            └───00002.jpg
            ...
        └───00001
            └───00001.jpg
            └───00002.jpg
            ...
        ...
    └───landmarks-dlib68
        └───00000
            └───00001.json
            └───00002.json
            ...
        └───00001
            └───00001.json
            └───00002.json
            ...
        ...
```

#### 2.1.5 將參考照片（reference image）放入 `/src/projects/fs_vid2vid/test_data/faceForensics/reference`。

比照 2.1.3 和 2.1.4，取得 reference image（e.g. `00000.jpg`）的臉部標記（e.g. `00000.json`），放入 `/reference` 內（只能放入一組）：
```
faceForensics
└───reference
    └───images
        └───00000.jpg
    └───landmarks-dlib68
        └───00000.json
└───driving
     ...
```

### 2.2 進行預測

In [ ]:
python inference.py --single_gpu --num_workers 0 \
--config configs/projects/fs_vid2vid/face_forensics/ampO1.yaml \
--output_dir projects/fs_vid2vid/output/face_forensics

### 2.3 檢視結果

In [ ]:
from ipywidgets import Video
Video.from_file("projects/fs_vid2vid/output/face_forensics/00000.mp4")